api call

In [29]:
from openai import OpenAI
from tqdm import tqdm
import httpx
import pandas as pd
import openai
import backoff

api_key = "sk-ShmyeH9VjAnRuT1S55A71a9fC69640948d20F73bA634C3A5"
client = OpenAI(
    base_url="https://oneapi.xty.app/v1",  # 中转url
    api_key=api_key,                      # api_key
    http_client=httpx.Client(
        proxies="http://127.0.0.1:7890"  # 代理地址
    ),
)

@backoff.on_exception(backoff.expo, (openai.APIStatusError, openai.InternalServerError), max_tries=5)
def get_responce(messages):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
    )
    return response.choices[0].message.content.strip('\n')


instruction1 = "given some log contents, the assistant should check if the following log contents share the same template.If so, abstract variables with ‘{placeholders}’ and return the template, otherwise return the templates and a list that represents the template used by each log"

instruction2 = "there is an exsiting log template: '{domain}:{port} error : A connection request was canceled before the completion.', the assistant should check if the following log contents share the same template I provided or you sh.If so, abstract variables with ‘{placeholders}’ and return the template, otherwise return the templates and a list that represents the template used by each log"

instruction0 = '''The AI assistant can parse some log contents to one or more templates: {"templates": [], "index": []}.The "templates" (list) field denotes the templates that the log contents use. The "index" (list) field denotes the index of template each log content use. You should abstract variables with ‘{placeholders}’ and only return the dictionary'''

instruction2 = '''Giving some log tempaltes, the AI assistant should merge the possibly same templates'''

instruction_noindex = '''by abstracting variables with {}, The AI assistant can parse some the giving log contents to most possibly one or two template. Only return the template or tempaltes without additonal explanation'''

prompt = '''Times: total = 1072, boot = 856, init = 210, finish = 6
Times: total = 1114, boot = 885, init = 223, finish = 6
Times: total = 1074, boot = 869, init = 199, finish = 6
Times: total = 1078, boot = 851, init = 219, finish = 8
Times: total = 1077, boot = 865, init = 206, finish = 6
Times: total = 38, boot = 11, init = 27, finish = 0
Times: total = 42, boot = 12, init = 30, finish = 0
Times: total = 41, boot = 15, init = 26, finish = 0
Times: total = 40, boot = 7, init = 33, finish = 0
Times: total = 42, boot = 13, init = 28, finish = 1
Times: total = 42, boot = -4131, init = 4172, finish = 1
Times: total = 41, boot = -4122, init = 4162, finish = 1
Times: total = 42, boot = -4121, init = 4162, finish = 1
Times: total = 42, boot = -4138, init = 4179, finish = 1
Times: total = 42, boot = -4134, init = 4175, finish = 1
Times: total = 39, boot = -147, init = 186, finish = 0
Times: total = 39, boot = -147, init = 186, finish = 0
Times: total = 41, boot = -139, init = 179, finish = 1
Times: total = 38, boot = -156, init = 194, finish = 0
Times: total = 39, boot = -159, init = 198, finish = 0
'''

prompt2 = '''Finished task {task} in stage {stage} (TID {ID}). {bytes} bytes result sent to driver
Finished task {task} in stage {stage} (TID {ID}). 2087 bytes result sent to driver
Finished task 3.0 in stage {stage} (TID {ID}). {bytes} bytes result sent to driver
Running task {task} in stage {stage} (TID {ID})'''

messages = [{"role": "system", "content": instruction_noindex}]
messages.append({"role": "user", "content": prompt})
print(get_responce(messages))

1. {total} = 1072, {boot} = 856, {init} = 210, {finish} = 6
2. {total} = 1114, {boot} = 885, {init} = 223, {finish} = 6
3. {total} = 1074, {boot} = 869, {init} = 199, {finish} = 6
4. {total} = 1078, {boot} = 851, {init} = 219, {finish} = 8
5. {total} = 1077, {boot} = 865, {init} = 206, {finish} = 6
6. {total} = 38, {boot} = 11, {init} = 27, {finish} = 0
7. {total} = 42, {boot} = 12, {init} = 30, {finish} = 0
8. {total} = 41, {boot} = 15, {init} = 26, {finish} = 0
9. {total} = 40, {boot} = 7, {init} = 33, {finish} = 0
10. {total} = 42, {boot} = 13, {init} = 28, {finish} = 1
11. {total} = 42, {boot} = -4131, {init} = 4172, {finish} = 1
12. {total} = 41, {boot} = -4122, {init} = 4162, {finish} = 1
13. {total} = 42, {boot} = -4121, {init} = 4162, {finish} = 1
14. {total} = 42, {boot} = -4138, {init} = 4179, {finish} = 1
15. {total} = 42, {boot} = -4134, {init} = 4175, {finish} = 1
16. {total} = 39, {boot} = -147, {init} = 186, {finish} = 0
17. {total} = 39, {boot} = -147, {init} = 186, {fi

Evaluate_gpt

In [1]:
from evaluator import evaluate
import pandas as pd
dataset = 'Linux'
file = f'outputs/enhanced_gpt/1shot/{dataset}.csv'
evaluate(file, dataset)

Precision: 0.9738, Recall: 0.5896, F1_measure: 0.7345, Group Accuracy: 0.2385, Message-Level Accuracy: 0.6090, Edit Distance: 1.4800


(0.2385, 0.609, 1.48, 2.9768439663509403)

验证按' '分词有多少词，以及含数字的词占多少

In [15]:
import re
import pandas as pd
datasets = ['BGL' ,'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC', 'Zookeeper', 'Mac','Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']
a = 0
b = 0
list = []
for dataset in datasets:
    df = pd.read_csv(f"dataset\{dataset}\{dataset}_2k.log_templates_corrected.csv")
    list = df['EventTemplate'].tolist()    
    total_words = 0
    words_with_numbers = 0

    for s in list:
        words = s.split()
        total_words += len(words)
        words_with_numbers += len([word for word in words if re.search(r'\d', word)])
    a+=total_words
    b+=words_with_numbers

print(f"{b} / {a}")

109 / 9841


In [2]:
import json
# load demonstrations
with open('demonstrations.json', 'r') as f:
    demonstrations = json.load(f)
messages = []
print(demonstrations['Spark'][0])

{'role': 'user', 'content': 'Log: Block broadcast_0 stored as values in memory (estimated size 384.0 B, free 317.5 KB)'}
